In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEndpoint
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HUGGINGFACEHUB_API_TOKEN"] = str(os.getenv("HUGGINGFACE_API_KEY"))
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = str(os.getenv("LANGCHAIN_API_KEY"))
os.environ["GROQ_API_KEY"] = str(os.getenv("Groq_Api_key"))
os.environ["TAVILY_API_KEY"] = str(os.getenv("tavily_api_key"))
# Load document_loaders

loader1 = WebBaseLoader("https://python.langchain.com/docs/tutorials/graph/")
loader2 = WebBaseLoader("https://python.langchain.com/docs/tutorials/agents/")
docs1 = loader1.load_and_split()
docs2 = loader2.load_and_split()

docs = [docs1 , docs2]

# Split documents

splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
splits = splitter.split_documents( docs2)

#Embeddings

embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# VectorStore

index = FAISS.from_documents(splits[:100], embed)

# retrieve

retriever = index.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.
c:\codehub\Rag Projects\project 2\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
retriever.get_relevant_documents("what is agent workflow?")

C:\Users\DELL 5300 A&I\AppData\Local\Temp\ipykernel_15780\2895562204.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("what is agent workflow?")


[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/agents/', 'title': 'Build an Agent | 🦜️🔗 LangChain', 'description': "By themselves, language models can't take actions - they just output text.", 'language': 'en'}, page_content="Agents are a complex topic with lots to learn!\nFor more information on Agents, please check out the LangGraph documentation. This has it's own set of concepts, tutorials, and how-to guides.Edit this pageWas this page helpful?PreviousBuild an Extraction ChainNextTaggingEnd-to-end agentSetupJupyter NotebookInstallationLangSmithTavilyDefine toolsUsing Language ModelsCreate the agentRun the agentStreaming MessagesStreaming tokensAdding in memoryConclusionCommunityTwitterGitHubOrganizationPythonJS/TSMoreHomepageBlogYouTubeCopyright © 2024 LangChain, Inc."),
 Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/agents/', 'title': 'Build an Agent | 🦜️🔗 LangChain', 'description': "By themselves, language models can't take 

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever , "langchain_retriver_tool" ,
                                       "You are a retriever tool that retrieve relevant documents from the database")

tavily = TavilySearchResults(max_results=1)
 
tools = [retriever_tool]


In [4]:
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

#llm
llm2 = ChatGroq(model = "llama-3.1-70b-versatile")

#prompt
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [5]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

agent = create_tool_calling_agent(llm2, tools, prompt)
agent_executer = AgentExecutor(agent=agent,tools=tools)

agent_executer.invoke({'input':'what is langchain?'})

{'input': 'what is langchain?',
 'output': 'LangChain is a system that supports the use of multiple language models interchangeably and allows for the creation of agents that can interact with various tools, such as a search engine. It provides a way to build conversational AI models that can perform tasks, make decisions, and take actions based on the input they receive. LangChain also includes tools for inspecting and understanding the inner workings of these models, making it easier to debug and improve their performance.'}

In [6]:
from typing import TypedDict , Annotated , List , Union
from langchain_core.agents import AgentAction , AgentFinish 
from langchain_core.messages import BaseMessage
import operator

class AgentState(TypedDict):
    # input string
    input:str
    # list of previus messeges in the conversation
    chat_history: list[BaseMessage]
    # outcome of given call to the agent
    # need 'none' as valid type , this is what this is start as
    agent_outcome: Union[AgentAction , AgentFinish , None]
    # list of action and intermediate steps 
    # Here we annotate with 'operater.add' to indicate that operater  
    # to this state should be added to the existing state (not overwritting it)
    intermediate_steps: Annotated[list[tuple[AgentAction, str ]] , operator.add]

In [7]:
from langgraph.prebuilt.tool_executor import ToolExecutor

tool_executer = ToolExecutor(tools)

#run agent
def run_agent(data):
    agent_outcome = agent.invoke(data)
    return {"agent_outcome": agent_outcome}

# define function to execute tools
def execute_tools(data):
    # get the most recent agent outcome - this is key added to the agent above
    agent_action = data["agent_outcome"]
    output = tool_executer.invoke(agent_action)
    return {"intermediate_steps": [(agent_action, str(output))]}

def should_continue(data):
    if isinstance(data["agent_outcome"],AgentFinish):
        return "End"
    else:
        return "Continue"

C:\Users\DELL 5300 A&I\AppData\Local\Temp\ipykernel_15780\3894753885.py:3: LangGraphDeprecationWarning: ToolExecutor is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_executer = ToolExecutor(tools)


In [8]:
from langgraph.graph import END , StateGraph

# langgraph workflows

workflow = StateGraph(AgentState)

#define the noes we will cylcle between
workflow.add_node("agent" , run_agent)
workflow.add_node("action", execute_tools)

workflow.set_entry_point("agent")

# define the edges between the nodes
workflow.add_conditional_edges(
    "agent" ,
    should_continue ,
    {
        "Continue": "action",
        "End": END
    }
)

workflow.add_edge("action" ,"agent")

# compile the workflow

app = workflow.compile()

In [9]:
app.invoke

<bound method Pregel.invoke of <langgraph.graph.state.CompiledStateGraph object at 0x0000028C141811F0>>

In [10]:
input = {"input":"what is langraph", "chat_history":[]}

# run the workflow
for s in app.stream(input):
    print(list(s.values())[0])
    print("--------------------------------")

{'agent_outcome': AgentFinish(return_values={'output': "LangGraph is not a widely recognized term. However, based on the context of the conversation, it seems like LangGraph might be related to LangChain, which is an open-source platform for building applications that involve large language models.\n\nLangChain provides a set of tools and APIs for working with language models, including support for multiple AI models, model serving, and integration with databases and other data sources. The platform is designed to make it easier to build and deploy applications that use large language models, such as chatbots, language translation systems, and text summarization tools.\n\nIf you have any specific questions about LangChain or its features, I'd be happy to try to help."}, log="LangGraph is not a widely recognized term. However, based on the context of the conversation, it seems like LangGraph might be related to LangChain, which is an open-source platform for building applications that i